In [1]:
import pandas as pd
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
import random
from deap import base, creator, tools

# ----------------------------
# 1. Load training and test data
# ----------------------------
# CSVs now have 9 columns per row:
#   Cols 0–4 = inputs,
#   Cols 5–6 = objectives,
#   Col  7 = ground span (must be ≤ 36),
#   Col  8 = air   span (must be ≤ 56)
train_df = pd.read_csv("../Trainingset_1000.csv")
# Drop any rows where the two original objectives are zero
train_df = train_df[(train_df.iloc[:, 5] != 0) & (train_df.iloc[:, 6] != 0)]

test_df  = pd.read_csv("../Testset_100.csv")  # same format

# Split into X / Y for the four outputs (two objectives + ground span + air span)
X_train      = train_df.iloc[:,   0:5].values.astype(np.float32)
Y1_train     = train_df.iloc[:,   5  ].values.astype(np.float32)     # Objective 1
Y2_train     = train_df.iloc[:,   6  ].values.astype(np.float32)     # Objective 2
Ground_train = train_df.iloc[:,   7  ].values.astype(np.float32)     # Ground span
Air_train    = train_df.iloc[:,   8  ].values.astype(np.float32)     # Air span

X_test       = test_df.iloc[:,    0:5].values.astype(np.float32)
Y1_test      = test_df.iloc[:,    5  ].values.astype(np.float32)
Y2_test      = test_df.iloc[:,    6  ].values.astype(np.float32)
Ground_test  = test_df.iloc[:,    7  ].values.astype(np.float32)
Air_test     = test_df.iloc[:,    8  ].values.astype(np.float32)

# Normalize inputs to [0,1] using training‐set min/max
X_min = X_train.min(axis=0)
X_max = X_train.max(axis=0)
X_train_norm = (X_train - X_min) / (X_max - X_min)
X_test_norm  = (X_test  - X_min) / (X_max - X_min)

# Convert to torch tensors
train_x       = torch.from_numpy(X_train_norm)
train_y1      = torch.from_numpy(Y1_train.reshape(-1, 1))
train_y2      = torch.from_numpy(Y2_train.reshape(-1, 1))
train_ground  = torch.from_numpy(Ground_train.reshape(-1, 1))
train_air     = torch.from_numpy(Air_train.reshape(-1, 1))

test_x       = torch.from_numpy(X_test_norm)
test_y1      = torch.from_numpy(Y1_test.reshape(-1, 1))
test_y2      = torch.from_numpy(Y2_test.reshape(-1, 1))
test_ground  = torch.from_numpy(Ground_test.reshape(-1, 1))
test_air     = torch.from_numpy(Air_test.reshape(-1, 1))

# ----------------------------
# 2. Define GP model class (unchanged)
# ----------------------------
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y.squeeze(), likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims=train_x.shape[1])
        )

    def forward(self, x):
        mean_x  = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# ----------------------------
# 3. Instantiate likelihoods and four GP models
# ----------------------------
likelihood1 = gpytorch.likelihoods.GaussianLikelihood()  # for Obj 1
likelihood2 = gpytorch.likelihoods.GaussianLikelihood()  # for Obj 2
likelihood3 = gpytorch.likelihoods.GaussianLikelihood()  # for Ground span
likelihood4 = gpytorch.likelihoods.GaussianLikelihood()  # for Air   span

model1 = ExactGPModel(train_x, train_y1,     likelihood1)  # Objective 1
model2 = ExactGPModel(train_x, train_y2,     likelihood2)  # Objective 2
model3 = ExactGPModel(train_x, train_ground, likelihood3)  # Ground span
model4 = ExactGPModel(train_x, train_air,    likelihood4)  # Air   span

# ----------------------------
# 4. Train each GP on the training set
# ----------------------------
def train_gp(model, likelihood, x_tr, y_tr, num_iter=300, lr=0.05):
    model.train()
    likelihood.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    for _ in range(num_iter):
        optimizer.zero_grad()
        output = model(x_tr)
        loss = -mll(output, y_tr.squeeze())
        loss.backward()
        optimizer.step()
    model.eval()
    likelihood.eval()

# Train all four GPs
train_gp(model1, likelihood1, train_x, train_y1)
train_gp(model2, likelihood2, train_x, train_y2)
train_gp(model3, likelihood3, train_x, train_ground)
train_gp(model4, likelihood4, train_x, train_air)

# ----------------------------
# 5. Helper to predict any GP’s mean
# ----------------------------
def predict_gp(model, likelihood, x_np):
    """
    x_np: numpy array of shape (n_points, 5) in original‐scale inputs.
    Returns: numpy array of shape (n_points,) = posterior mean.
    """
    x_norm = (x_np - X_min) / (X_max - X_min)
    xt = torch.from_numpy(x_norm.astype(np.float32))
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        preds = likelihood(model(xt))
        return preds.mean.cpu().numpy()

# ----------------------------
# 6. Define GA bounds for the five inputs only
# ----------------------------
BOUND_LOW  = [  12.00,  0.5,   5.00, 0.50,  0.00 ]
BOUND_HIGH = [ 22.00,  1.00,  35.00, 0.85, 40.00 ]

# Penalty value if either constraint is violated
PENALTY = 1e6

# ----------------------------
# 7. GA for Objective 1 with both constraints: ground ≤ 36 AND air ≤ 56
# ----------------------------
creator.create("FitnessMin1", base.Fitness, weights=(-1.0,))  # minimize Obj1 + penalty
creator.create("Individual1", list, fitness=creator.FitnessMin1)

toolbox1 = base.Toolbox()
# Register five uniform‐float generators (one per input dimension)
for i in range(5):
    lo, hi = BOUND_LOW[i], BOUND_HIGH[i]
    toolbox1.register(f"attr_float1_{i}", random.uniform, lo, hi)

toolbox1.register(
    "individual",
    tools.initCycle,
    creator.Individual1,
    [toolbox1.attr_float1_0,
     toolbox1.attr_float1_1,
     toolbox1.attr_float1_2,
     toolbox1.attr_float1_3,
     toolbox1.attr_float1_4],
    n=1
)
toolbox1.register("population", tools.initRepeat, list, toolbox1.individual)

def evaluate_obj1_both_constraints(individual):
    """
    individual: list of 5 floats = [ x1 … x5 ] in original scale.
    1) Predict ground span with model3.
    2) Predict air   span with model4.
    3) If ground_pred > 36 OR air_pred > 56 → return (PENALTY,).
    4) Else → return (predict_obj1,).
    """
    x_orig = np.array(individual, dtype=np.float32).reshape(1, -1)  # (1,5)
    ground_pred = predict_gp(model3, likelihood3, x_orig)[0]
    air_pred    = predict_gp(model4, likelihood4, x_orig)[0]
    if (ground_pred > 36.0) or (air_pred > 56.0):
        return (PENALTY,)
    f1_pred = predict_gp(model1, likelihood1, x_orig)[0]
    return (float(f1_pred),)

toolbox1.register("evaluate", evaluate_obj1_both_constraints)

# Bounded SBX crossover and polynomial mutation (same as before)
ETA_CX   = 20.0
ETA_MUT  = 20.0
MUT_PROB = 1.0 / 5.0

toolbox1.register(
    "mate",
    tools.cxSimulatedBinaryBounded,
    low=BOUND_LOW,
    up=BOUND_HIGH,
    eta=ETA_CX
)
toolbox1.register(
    "mutate",
    tools.mutPolynomialBounded,
    low=BOUND_LOW,
    up=BOUND_HIGH,
    eta=ETA_MUT,
    indpb=MUT_PROB
)
toolbox1.register("select", tools.selTournament, tournsize=3)

def run_ga_obj1(n_pop=200, n_gen=300, cxpb=0.9, mutpb=0.2):
    pop = toolbox1.population(n=n_pop)
    # Evaluate initial population
    for ind in pop:
        ind.fitness.values = toolbox1.evaluate(ind)
    for gen in range(1, n_gen + 1):
        # Selection
        offspring = toolbox1.select(pop, len(pop))
        offspring = list(map(toolbox1.clone, offspring))
        # Crossover
        for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
            if random.random() <= cxpb:
                toolbox1.mate(ind1, ind2)
                del ind1.fitness.values
                del ind2.fitness.values
        # Mutation
        for mutant in offspring:
            if random.random() <= mutpb:
                toolbox1.mutate(mutant)
                del mutant.fitness.values
        # Re‐evaluate any invalid (new) individuals
        invalid = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid:
            ind.fitness.values = toolbox1.evaluate(ind)
        pop = offspring
    return tools.selBest(pop, k=1)[0]

best1 = run_ga_obj1()
x1_best  = np.array(best1, dtype=np.float32).tolist()
ground1  = predict_gp(model3, likelihood3, np.array(x1_best).reshape(1, -1))[0]
air1     = predict_gp(model4, likelihood4, np.array(x1_best).reshape(1, -1))[0]
f1_best  = predict_gp(model1, likelihood1, np.array(x1_best).reshape(1, -1))[0]
f2_pred  = predict_gp(model2, likelihood2, np.array(x1_best).reshape(1, -1))[0]

print("\nOptimal for Objective 1 subject to ground_span ≤ 36 AND air_span ≤ 56:")
print(f"  Inputs (orig scale): {x1_best}")
print(f"  Predicted ground span: {ground1:.3f}  (≤ 36)")
print(f"  Predicted air   span: {air1:.3f}  (≤ 56)")
print(f"  Predicted Obj1       : {f1_best:.6f}")
print(f"  Predicted Obj2       : {f2_pred:.6f}")

# ----------------------------
# 8. GA for Objective 2 with both constraints
# ----------------------------
creator.create("FitnessMin2", base.Fitness, weights=(-1.0,))
creator.create("Individual2", list, fitness=creator.FitnessMin2)

toolbox2 = base.Toolbox()
for i in range(5):
    lo, hi = BOUND_LOW[i], BOUND_HIGH[i]
    toolbox2.register(f"attr_float2_{i}", random.uniform, lo, hi)

toolbox2.register(
    "individual",
    tools.initCycle,
    creator.Individual2,
    [toolbox2.attr_float2_0,
     toolbox2.attr_float2_1,
     toolbox2.attr_float2_2,
     toolbox2.attr_float2_3,
     toolbox2.attr_float2_4],
    n=1
)
toolbox2.register("population", tools.initRepeat, list, toolbox2.individual)

def evaluate_obj2_both_constraints(individual):
    x_orig     = np.array(individual, dtype=np.float32).reshape(1, -1)
    ground_pred = predict_gp(model3, likelihood3, x_orig)[0]
    air_pred    = predict_gp(model4, likelihood4, x_orig)[0]
    if (ground_pred > 36.0) or (air_pred > 56.0):
        return (PENALTY,)
    f2_pred = predict_gp(model2, likelihood2, x_orig)[0]
    return (float(f2_pred),)

toolbox2.register("evaluate", evaluate_obj2_both_constraints)

toolbox2.register(
    "mate",
    tools.cxSimulatedBinaryBounded,
    low=BOUND_LOW,
    up=BOUND_HIGH,
    eta=ETA_CX
)
toolbox2.register(
    "mutate",
    tools.mutPolynomialBounded,
    low=BOUND_LOW,
    up=BOUND_HIGH,
    eta=ETA_MUT,
    indpb=MUT_PROB
)
toolbox2.register("select", tools.selTournament, tournsize=3)

def run_ga_obj2(n_pop=200, n_gen=300, cxpb=0.9, mutpb=0.2):
    pop = toolbox2.population(n=n_pop)
    for ind in pop:
        ind.fitness.values = toolbox2.evaluate(ind)
    for gen in range(1, n_gen + 1):
        offspring = toolbox2.select(pop, len(pop))
        offspring = list(map(toolbox2.clone, offspring))
        for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
            if random.random() <= cxpb:
                toolbox2.mate(ind1, ind2)
                del ind1.fitness.values
                del ind2.fitness.values
        for mutant in offspring:
            if random.random() <= mutpb:
                toolbox2.mutate(mutant)
                del mutant.fitness.values
        invalid = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid:
            ind.fitness.values = toolbox2.evaluate(ind)
        pop = offspring
    return tools.selBest(pop, k=1)[0]

best2 = run_ga_obj2()
x2_best  = np.array(best2, dtype=np.float32).tolist()
ground2  = predict_gp(model3, likelihood3, np.array(x2_best).reshape(1, -1))[0]
air2     = predict_gp(model4, likelihood4, np.array(x2_best).reshape(1, -1))[0]
f2_best  = predict_gp(model2, likelihood2, np.array(x2_best).reshape(1, -1))[0]
f1_pred  = predict_gp(model1, likelihood1, np.array(x2_best).reshape(1, -1))[0]

print("\nOptimal for Objective 2 subject to ground_span ≤ 36 AND air_span ≤ 56:")
print(f"  Inputs (orig scale): {x2_best}")
print(f"  Predicted ground span: {ground2:.3f}  (≤ 36)")
print(f"  Predicted air   span: {air2:.3f}  (≤ 56)")
print(f"  Predicted Obj1       : {f1_pred:.6f}")
print(f"  Predicted Obj2       : {f2_best:.6f}")



Optimal for Objective 1 subject to ground_span ≤ 36 AND air_span ≤ 56:
  Inputs (orig scale): [22.0, 0.5, 35.0, 0.6173838973045349, 4.767297667740422e-08]
  Predicted ground span: 24.630  (≤ 36)
  Predicted air   span: 47.827  (≤ 56)
  Predicted Obj1       : 7587.261230
  Predicted Obj2       : 0.007712

Optimal for Objective 2 subject to ground_span ≤ 36 AND air_span ≤ 56:
  Inputs (orig scale): [12.00001335144043, 0.5000067353248596, 34.99999237060547, 0.8499999642372131, 3.32091876771301e-05]
  Predicted ground span: 18.079  (≤ 36)
  Predicted air   span: 34.887  (≤ 56)
  Predicted Obj1       : 9571.231445
  Predicted Obj2       : 0.006175
